In [16]:
import numpy as np
import cvxpy as cp
from sdp_data_matrix_scalable import smooth_convex_gd, generate_sample

In [17]:
d = 5
K = 1
mu = 0.0
L = 1.0
gamma = 1/L

A, b, C = smooth_convex_gd(d, K, mu, L, gamma)
b = np.array(b)

#### One sample $(N = 1)$ and linear loss $\ell$ $(J = 1)$

In [18]:
sample_n = 1
sample_xks, sample_gks, sample_fks = generate_sample(d, K, mu, L, gamma, seed=0)
sample_P = sample_xks[0]
for g in sample_gks[:-1] :
    sample_P = np.concatenate([sample_P, g], axis=1)
sample_G = sample_P.T @ sample_P
sample_F = np.concatenate(sample_fks, axis=1)[0]
print(sample_G)
print(sample_F)

[[1.         0.7613128 ]
 [0.7613128  0.65023841]]
[0.3806564 0.       ]


In [19]:
G_shape = C.G.shape
F_shape = C.F.shape
m = len(b)
G_shape, F_shape, m

((2, 2), (2,), 5)

In [20]:
sample_G.shape, sample_F.shape

((2, 2), (2,))

In [36]:
lmbd = cp.Variable()
s = cp.Variable()
G = cp.Variable(G_shape, symmetric=True)
F = cp.Variable(F_shape)
y = cp.Variable(m)
Aopt = -C.G
bopt = -C.F
c = -b

constraint = [- c.T@y - cp.trace(G@sample_G) - F.T@sample_F <= s]
constraint += [cp.SOC(lmbd, cp.hstack([cp.vec(G), F]))]
LstarG = np.zeros(G_shape)
LstarF = np.zeros(F_shape)
for i in range(m) :
    LstarG = LstarG + y[i]*A[i].G
    LstarF = LstarF + y[i]*A[i].F
constraint += [LstarG - G - Aopt >> 0]
constraint += [LstarF - F - bopt == 0]
constraint += [y >= 0]

print(Aopt, bopt, c)
print(m)
print(sample_G)
print(sample_F)
for i in range(m):
    print(A[i].G, A[i].F)
print(len(constraint))

[[-0. -0.]
 [-0. -0.]] [1. 0.] [-0. -0. -0. -0. -1.]
5
[[1.         0.7613128 ]
 [0.7613128  0.65023841]]
[0.3806564 0.       ]
[[0.  0. ]
 [0.  0.5]] [-1.  0.]
[[ 0.  -0.5]
 [-0.5  0.5]] [1. 0.]
[[0.  0. ]
 [0.  0.5]] [-1.  0.]
[[ 0.  -0.5]
 [-0.5  0.5]] [1. 0.]
[[1. 0.]
 [0. 0.]] [0. 0.]
5


##### When `eps=0.0`, it is the same as the sample convergence bound. When `eps` super large, it is the same as the PEP bound.

In [31]:
# eps = 0.0 (sample) <-------> eps = 1e10 (PEP)
eps = 0.2
prob = cp.Problem(cp.Minimize(lmbd*eps + 1/sample_n * cp.sum(s)), constraint)
prob.solve(solver=cp.CLARABEL)
print('optimal value (DRO-PEP) =', prob.value)
print('cvxpy status =', prob.status)
# print(lmbd.value)
# print(s.value)

optimal value (DRO-PEP) = 0.49030169441124394
cvxpy status = optimal


In [23]:
sample_F

array([0.3806564, 0.       ])

#### compare with PEP result

In [24]:
G_PEP = cp.Variable(C.G.shape, symmetric=True)
F_PEP = cp.Variable(C.F.shape)

constraint_PEP = [G_PEP >> 0]
constraint_PEP += [F_PEP >= 0.0]
for i in range(m) :
    constraint_PEP += [cp.trace(G_PEP@A[i].G) + F_PEP.T@A[i].F <= b[i]]
prob = cp.Problem(cp.Maximize(- cp.trace(G_PEP@C.G) - F_PEP.T@C.F), constraint_PEP)
prob.solve(solver=cp.CLARABEL)
print('optimal value (PEP) =', prob.value)
print('cvxpy status =', prob.status)

optimal value (PEP) = 0.4999999950355775
cvxpy status = optimal
